In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_excel('Huang_Massa_data_with_all_SMILES.xlsx')
df.head()

,num,Molecular Name,Formula,Density (g/cm3),Delta Hf solid (kj/mol),Explosive energy (kj/cc),Shock velocity (km/s),Particle velocity (km/s),Speed of sound (km/s),Pressure (Gpa),T(K),TNT Equiv (per cc),group,SMILES,NOTE
0,1,CUGCOW,C8N7O14H1,2.024,641.16,6.419,9.569,2.519,7.050,48.777,5440,2.233,Cubane,O=N(=O)C12C3C4(N(=O)=O)C1(N(=O)=O)C1(N(=O)=O)C...,NaN
1,2,PUBMUU02,C6N12O12H6,2.044,430.83,5.623,9.972,2.346,7.626,47.807,3886,1.828,CL20,O=N(=O)N1C2C3N(N(=O)=O)C4C1N(N(=O)=O)C(C(N2N(=...,NaN
2,3,KURT08MO,C4N8O10H4F4,2.028,-151.80,5.290,8.365,2.286,6.079,38.780,4342,1.842,Linear,[H]C([H])(N(N(=O)=O)C([H])([H])C(N(F)F)(N(=O)=...,NaN
3,4,ELUDEV,C4N10O6H2,1.996,657.13,5.228,9.729,2.289,7.441,44.490,3950,1.698,Pyrazole,O=N(=O)NC1=NON=C1N=NC1=NON=C1NN(=O)=O,NaN
4,5,UGUGUY,C8N6O12H2,1.931,593.87,5.188,9.061,2.318,6.743,40.555,5184,1.920,Cubane,O=N(=O)C12C3C4(C1C1(N(=O)=O)C2(N(=O)=O)C3(N(=O...,NaN


In [3]:
def rand_generator(smi,y):
    smiles_list = []
    y_list = []
    for _ in range(70):
        mol = Chem.MolFromSmiles(smi)
        if not mol:
            return None
        smiles_list.append(Chem.MolToSmiles(
            mol, canonical = False, doRandom = True, isomericSmiles = False, kekuleSmiles = False 
        ))

    smiles_list.append(smi)
    smiles_list = list(set(filter(None, smiles_list)))
    for _ in range(len(smiles_list)):
        y_list.append(y)
    y_list = list(filter(None, y_list))    
    return smiles_list, y_list

In [4]:
smiles = list(df.SMILES)
y = list(df['Explosive energy (kj/cc)'])
outcome = map(rand_generator, smiles, y)
outcome =list(outcome)
outcome_final = list(filter(None, outcome))
outcome_final_array = np.array(outcome_final)
final_dict = pd.DataFrame(columns=['SMILES', 'Y'])

<ipython-input-4-a9f2a51f61ab>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  outcome_final_array = np.array(outcome_final)


In [5]:
for i in range(len(outcome_final)):
    for m in range(len(outcome_final[i][0])):
        d = {'SMILES': outcome_final_array[i][0][m], 'Y': outcome_final_array[i][1][m]}
        final_dict = final_dict.append(d, ignore_index=True)

In [6]:
final_dict.to_csv('augmented_smiles.csv')